## 1. volatile 的作用
- Synchronized 是一个比较重量级的操作，对系统的性能有比较大的影响
- volatile 关键字就是 Java 中提供的另一种解决可见性和有序性问题的方案
- 对于原子性，需要强调一点，也是大家容易误解的一点：对 volatile 变量的单次读/写操作可以保证原子性的，如 long 和 double 类型变量，但是并不能保证 i++ 这种操作的原子性，因为本质上 i++ 是读、写两次操作

---

## 2. volatile 的使用
### 2.1 防止重排序
- 从一个最经典的例子来分析重排序问题。大家应该都很熟悉单例模式的实现，而在并发环境下的单例实现方式，我们通常可以采用双重检查加锁（DCL）的方式来实现。

```
package com.paddx.test.concurrent;

public class Singleton {
    public static volatile Singleton singleton;

    /**
     * 构造函数私有，禁止外部实例化
     */
    private Singleton() {};

    public static Singleton getInstance() {
        if (singleton == null) {
            synchronized (singleton) {
                if (singleton == null) {
                    singleton = new Singleton();
                }
            }
        }
        return singleton;
    }
}
```

#### 为什么要在变量 singleton 之间加上 volatile 关键字
- 要理解这个问题，先要了解对象的构造过程，实例化一个对象其实可以分为三个步骤:
  - 分配内存空间
  - 初始化对象
  - 将内存空间的地址赋值给对应的引用
  
- 但是由于操作系统可以对指令进行重排序，所以上面的过程也可能会变成如下过程：
  - 分配内存空间
  - 将内存空间的地址赋值给对应的引用
  - 初始化对象

- 如果是这个流程，多线程环境下就可能将一个未初始化的对象引用暴露出来，从而导致不可预料的结果。因此，为了防止这个过程的重排序，我们需要将变量设置为 volatile 类型的变量。

---

### 2.2 实现可见性
- 可见性问题主要指一个线程修改了共享变量值，而另一个线程却看不到
- 引起可见性问题的主要原因是每个线程拥有自己的一个高速缓存区 —— 线程工作内存

```
package com.paddx.test.concurrent;

public class VolatileTest {
    int a = 1;
    int b = 2;

    public void change() {
        a = 3;
        b = a;
    }

    public void print(){
        System.out.println("b=" + b + ";a=" + a);
    }

    public static void main(String[] args) {
        while (true) {
            final VolatileTest test = new VolatileTest();
            new Thread(new Runnable() {
                @Override
                public void run() {
                    try {
                        Thread.sleep(10);
                    } catch (InterruptedException e) {
                        e.printStackTrace();
                    }
                    test.change();
                }
            }).start();

            new Thread(new Runnable() {
                @Override
                public void run() {
                    try {
                        Thread.sleep(10);
                    } catch (InterruptedException e) {
                        e.printStackTrace();
                    }
                    test.print();
                }
            }).start();
        }
    }
}
```

- 这段代码的结果只可能有两种：b=3;a=3 或 b=2;a=1。
- 实际上会出现第三种场景：b=3;a=1
  - 原因就是第一个线程将值 a=3 修改后，但是对第二个线程是不可见的，所以才出现这一结果。如果将 a 和 b 都改成 volatile 类型的变量再执行，则再也不会出现 b=3;a=1 的结果了。

---

### 2.3 保证原子性
- 因为 long 和 double 两种数据类型的操作可分为高32位和低32位两部分，因此普通的 long 或 double 类型读/写可能不是原子的。因此，鼓励大家将共享的 long 和 double 变量设置为 volatile 类型，这样能保证任何情况下对 long 和 double 的单次读/写操作都具有原子性。

```
package com.paddx.test.concurrent;

public class VolatileTest01 {
    volatile int i;

    public void addI() {
        i++;
    }

    public static void main(String[] args) throws InterruptedException {
        final  VolatileTest01 test01 = new VolatileTest01();
        for (int n = 0; n < 1000; n++) {
            new Thread(new Runnable() {
                @Override
                public void run() {
                    try {
                        Thread.sleep(10);
                    } catch (InterruptedException e) {
                        e.printStackTrace();
                    }
                    test01.addI();
                }
            }).start();
        }

        Thread.sleep(10000);  // 等待10秒，保证上面程序执行完成

        System.out.println(test01.i);
    }
}
```

- 可以自己尝试下，每个人运行的结果不相同。volatile 是无法保证原子性的（否则结果应该是1000）。原因也很简单，i++ 其实是一个复合操作，包括三步骤：
  - 读取 i 的值
  - 对 i 加 1
  - 将 i 的值写回内存

- volatile 是无法保证这三个操作是具有原子性的，我们可以通过 AtomicInteger 或者 Synchronized 来保证 +1 操作的原子性。

---

## 3. volatile 的原理
### 3.1 可见性实现
- 线程本身并不直接与主内存进行数据的交互，而是通过线程的工作内存来完成相应的操作。这也是导致线程间数据不可见的本质原因。因此要实现 volatile 变量的可见性，直接从这方面入手即可。
- 对 volatile 变量的写操作与普通变量的主要区别有两点：
  - 修改 volatile 变量时会强制将修改后的值刷新到主内存中
  - 修改 volatile 变量后会导致其他线程工作内存中对应的变量值失效。因此，再读取该变量值的时候就需要重新从读取主内存中的值
  
### 3.2 有序性实现
- happen-before 规则(JSR 133中对Happen-before的定义)
  - 如果 a happen-before b，则 a 所做的任何操作对 b 是可见的。（这一点大家务必记住，因为 happen-before 这个词容易被误解为是时间的前后）
  
- JSR 133 中定义了哪些 happen-before 规则：
  - 同一个线程中的，前面的操作 happen-before 后续的操作。（即单线程内按代码顺序执行。但是，在不影响在单线程环境执行结果的前提下，编译器和处理器可以进行重排序，这是合法的。换句话说，这一是规则无法保证编译重排和指令重排）。
  - 监视器上的解锁操作 happen-before 其后续的加锁操作。（Synchronized 规则）
  - 对 volatile 变量的写操作 happen-before 后续的读操作。（volatile 规则）
  - 线程的 start() 方法 happen-before 该线程所有的后续操作。（线程启动规则）
  - 线程所有的操作 happen-before 其他线程在该线程上调用 join 返回成功后的操作
  - 如果 a happen-before b，b happen-before c，则 a happen-before c（传递性）

- JMM 针对 volatile变量所规定的重排序规则表：
![image](page1.png)

### 3.3 内存屏障
- 为了实现 volatile 可见性和 happen-befor 的语义。JVM 底层是通过一个叫做“内存屏障”的东西来完成。内存屏障，也叫做内存栅栏，是一组处理器指令，用于实现对内存操作的顺序限制

![image](page2.png)

#### 3.3.1 LoadLoad 屏障
- 执行顺序：Load1 —> Loadload —> Load2
- 确保 Load2 及后续 Load 指令加载数据之前能访问到 Load1 加载的数据

#### 3.3.2 StoreStore 屏障
- 执行顺序：Store1 —> StoreStore —> Store2
- 确保 Store2 以及后续 Store 指令执行前，Store1 操作的数据对其它处理器可见

#### 3.3.3. LoadStore 屏障
- 执行顺序： Load1 —> LoadStore —> Store2
- 确保 Store2 和后续 Store 指令执行前，可以访问到 Load1 加载的数据

#### 3.3.4 StoreLoad 屏障
- 执行顺序: Store1 —> StoreLoad —> Load2
- 确保 Load2 和后续的 Load 指令读取之前，Store1 的数据对其他处理器是可见的

```
package com.paddx.test.concurrent;

public class MemoryBarrier {
    int a, b;
    volatile int v, u;

    void f() {
        int i, j;

        i = a;
        j = b;
        i = v;
        
        // LoadLoad
        j = u;
        
        // LoadStore
        a = i;
        b = j;
        
        // StoreStore
        v = i;
        
        // StoreStore
        u = j;
        
        // StoreLoad
        i = u;
        
        // LoadLoad
        // LoadStore
        j = b;
        a = i;
    }
}
```

---

## 4. 总结
- volatile 是并发编程中的一种优化，在某些场景下可以代替 Synchronized。但是，volatile 不能完全取代 Synchronized 的位置，只有在一些特殊的场景下，才能适用 volatile。
- 必须同时满足下面两个条件才能保证在并发环境的线程安全：
  - 对变量的写操作不依赖于当前值
  - 该变量没有包含在具有其他变量的不变式中

---